In [ ]:
# 이산 웨이블릿 변환(DWT) 코드 실행에 필요한 라이브러리
import pandas as pd
from matplotlib import pyplot as plt
import pywt
import os
import tqdm 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from numpy import arange, ones, pi
from scipy import cos, sin
from scipy.fftpack import fft, fftfreq, ifft
from scipy.stats import skew
import pywt
from scipy.stats import kurtosis

In [ ]:
# 파일이 저장된 디렉토리
abspath = os.path.abspath('./dataset')

In [ ]:
# 이산 웨이블릿 변환하는 함수
def dwt(data):
    eli_len = len(data) % 64
    data = data.iloc[:-eli_len]
        
    y_hori = data['Horiz'] 
    y_verti = data['Vert'] 
            
    h_coff = pywt.swt(list(y_hori), 'db4',level=5,axis=0)
    v_coff = pywt.swt(list(y_verti), 'db4',level=5,axis = 0)
    
    
    # concat wavelet coff
    h_data = pd.concat([pd.DataFrame(i).T for i in h_coff],axis =1)
    v_data = pd.concat([pd.DataFrame(i).T for i in v_coff],axis =1)
    
    # set col name
    col_name_h = ['h_cA5','h_cD5','h_cA4','h_cD4','h_cA3','h_cD3','h_cA2','h_cD2','h_cA1','h_cD1']
    col_name_v = ['v_cA5','v_cD5','v_cA4','v_cD4','v_cA3','v_cD3','v_cA2','v_cD2','v_cA1','v_cD1']

    h_data.columns = col_name_h
    v_data.columns = col_name_v
    
    # merge(기본, H coffs , V coffs)
    final = pd.concat([data,h_data,v_data],axis= 1)
    final_value = final.rolling(5).mean().max()
    return final_value

In [ ]:
import time
def fix_file(df):
    df = pd.Series(df[df.columns[0]])
    #데이터 형식이 안맞는 경우 
    fixed = pd.DataFrame()
    fixed['H'] = df.str.split(';').str[0]
    fixed['M'] = df.str.split(';').str[1]
    fixed['S'] = df.str.split(';').str[2]
    fixed['MS'] = df.str.split(';').str[3]
    fixed['Horiz'] = df.str.split(';').str[4]
    fixed['Vert'] = df.str.split(';').str[5]
    return fixed

# csv파일을 읽어와 이산 웨이블릿 변환을 적용하는 코드
def read_data(path):
    file_list = os.listdir(path)
    total_list = []
    for i in tqdm.tqdm(range(len(file_list[:-1]))):
        
        tem = pd.read_csv(path+'/'+file_list[i])
        if len(tem.columns) == 6:
            tem.columns = ['H','M','S','MS','Horiz','Vert']
            
            five_point_results = dwt(tem)
            total_list.append(five_point_results)
            
            
        if len(tem.columns) == 1:
            tem2 = fix_file(tem)
            tem2.columns = ['H','M','S','MS','Horiz','Vert']
            
            five_point_results = dwt(tem2)
            total_list.append(five_point_results)        
            
        else:
            continue
            
    return total_list 


start = time.time()
files = os.listdir(abspath)
files

# TEST/LEARNING (파일명)
for get_file in files:
    path_dir = abspath+"/"+get_file
    file2 = os.listdir(path_dir)
    # Bearing file 접근
    for file in file2:
        if file !='Bearing1_1':
          continue
        path2 = path_dir+'/'+file
        tem_fft = read_data(path2)
        fft_data = pd.DataFrame(tem_fft)
        # CSV 저장
        fft_data=fft_data.sort_values(by=['H','M','S','MS'])
        fft_data = fft_data.reset_index(drop=True)     
        fft_data.to_csv('./'+file+"_dwt.csv",index=False)

        # PLOT 결과 저장
        plt.title(file+"top5")
        plt.xlabel("Observation #")
        plt.ylabel("Feature (g)")
        print(fft_data.head())
        plt.plot(fft_data['Horiz'],label='Horizontal')
        plt.plot(fft_data['Vert'],label='Vertical')

        plt.legend(loc = 'upper right')
        plt.savefig('./dwt/'+file+'_top5.png')        
        plt.close()
        print("================="+file+" DONE ================")
        
end = time.time()
print("변환 소요 시간: ",f"{end - start:.5f} sec")

['Learning_set', 'Test_set']
Full_Test_Set
Bearing2_1
Bearing3_2
Bearing2_2
Bearing3_1
Bearing1_2
Bearing1_1


100%|██████████| 2802/2802 [28:04<00:00,  1.66it/s]


     H     M     S        MS   Horiz    Vert     h_cA5     h_cD5     h_cA4  \
0  9.0  38.0  46.0  963084.0  3.7892  0.7042  5.757505  8.546910  7.569509   
1  9.0  38.0  46.0  963084.0  2.8348  0.5402  5.821120  5.340468  6.069265   
2  9.0  39.0  39.0  163084.0  1.0764  0.5596  1.049991  0.922511  1.114473   
3  9.0  39.0  49.0  163084.0  0.9760  0.5398  1.156439  0.908138  1.112332   
4  9.0  39.0  59.0  163084.0  0.7118  0.4946  0.929293  0.937082  0.943502   

      h_cD4  ...     v_cA5     v_cD5     v_cA4     v_cD4     v_cA3     v_cD3  \
0  8.420201  ...  1.164863  1.561817  1.179080  1.628315  1.183051  0.898459   
1  6.411421  ...  1.140003  1.200478  1.143382  1.080769  1.017019  0.755240   
2  1.330087  ...  0.475693  0.405332  0.412344  0.657089  0.562914  1.017333   
3  1.725001  ...  0.551952  0.356781  0.495348  0.790310  0.683713  1.035748   
4  1.514617  ...  0.571178  0.379509  0.559000  0.620733  0.509998  0.948676   

      v_cA2     v_cD2     v_cA1     v_cD1  
0  1.1